In [1]:
import pandas as pd
import pickle

In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import glm

In [3]:
from sklearn.preprocessing import StandardScaler


In [4]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#finalgen_samples = pd.read_csv('../final_gen.csv')['sample_name']
first_gen = pd.read_csv('../key_files/generation_1_sample_names.txt',header=None)[0]
samples = first_gen.to_list()

#clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bio')
clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_sites_era5_year_2018.csv')

sites_af = pd.Series(samples).str.split('_').str[0].astype(int)

sites_af.name = 'site'

env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)

In [6]:
env_variable = env['bio1']

In [7]:
# Standardize the environmental variable
scaler = StandardScaler()
env_variable_scaled = scaler.fit_transform(env_variable.values.reshape(-1, 1))

In [8]:
pd.DataFrame(env_variable_scaled).to_csv('env.csv',index=None)

In [9]:
env_site_scaled = pd.concat([env['site'],pd.Series(env_variable_scaled.flatten())],axis=1)

In [10]:
env_site_scaled.columns = ['site', 'env_scaled']

In [11]:
env_site_scaled

,site,env_scaled
0,1,-0.795347
1,1,-0.795347
2,1,-0.795347
3,1,-0.795347
4,1,-0.795347
...,...,...
321,60,1.343429
322,60,1.343429
323,60,1.343429
324,60,1.343429


In [74]:
env_site_scaled.to_csv('env_site_scaled.csv',index=None)

In [75]:
env_site_scaled

,site,env_scaled
0,1,-0.795347
1,1,-0.795347
2,1,-0.795347
3,1,-0.795347
4,1,-0.795347
...,...,...
321,60,1.343429
322,60,1.343429
323,60,1.343429
324,60,1.343429


In [ ]:
'results/split_29/partition0.csv

In [76]:
file_path = '../jacknife_first_gen/splits_samples_first_gen.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

In [77]:
split = 29
partition = 0

In [78]:
train = samples[split][0]
test = samples[split][1]

In [79]:
##get the sites to train/test 
sites_u_train = pd.Series(train).str.split('_').str[0].unique()
sites_u_test = pd.Series(test).str.split('_').str[0].unique()
# conver tthem to int for easiest manipulations
sites_u_train_int =  [int(i) for i in sites_u_train ]
sites_u_test_int =  [int(i) for i in sites_u_test ]

In [80]:
env_site_scaled = pd.read_csv('env_site_scaled.csv')
env_variable_train = env_site_scaled[env_site_scaled['site'].isin(sites_u_train_int)]['env_scaled'].reset_index(drop=True)

In [81]:
env_variable_train

0     -0.795347
1     -0.795347
2     -0.795347
3     -0.795347
4     -0.795347
         ...   
309    1.343429
310    1.343429
311    1.343429
312    1.343429
313    1.343429
Name: env_scaled, Length: 314, dtype: float64

In [82]:
env_variable_test = env_site_scaled[env_site_scaled['site'].isin(sites_u_test_int)]
env_site_scaled_unique = env_variable_test.drop_duplicates().set_index('site')


In [83]:
partition_0 = pd.read_csv(f'../baypass_first_gen/individual_gfiles/partition_{partition}.txt',header=None) 

pickle_file_path = f'../baypass_first_gen/individual_gfiles/column_names_partition_{partition}'
with open(pickle_file_path, 'rb') as file:
    data0 = pickle.load(file)

In [84]:
partition_0.columns = data0

In [85]:
partition_0

,1_1_1_minor,1_1_1_major,1_1_2_minor,1_1_2_major,1_1_3_minor,1_1_3_major,1_1_4_minor,1_1_4_major,1_1_5_minor,1_1_5_major,...,60_1_8_minor,60_1_8_major,60_1_9_minor,60_1_9_major,60_1_10_minor,60_1_10_major,60_1_11_minor,60_1_11_major,60_1_12_minor,60_1_12_major
0,89,203,114,356,124,334,122,344,159,293,...,11,37,9,5,10,32,19,21,24,16
1,14,278,32,438,27,431,32,434,45,407,...,7,41,0,14,0,42,3,37,1,39
2,45,247,55,415,53,405,41,425,31,421,...,11,37,0,14,1,41,4,36,1,39
3,14,278,10,460,26,432,26,440,29,423,...,7,41,1,13,8,34,7,33,4,36
4,14,278,10,460,26,432,26,440,29,423,...,7,41,1,13,8,34,7,33,4,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,43,249,60,410,75,383,62,404,64,388,...,7,41,5,9,10,32,1,39,2,38
5220,91,201,200,270,146,312,169,297,157,295,...,9,39,5,9,20,22,14,26,23,17
5221,28,264,44,426,53,405,38,428,41,411,...,7,41,2,12,0,42,0,40,1,39
5222,98,194,201,269,154,304,170,296,159,293,...,9,39,5,9,21,21,14,26,23,17


In [86]:
sites_u_train

array(['1', '2', '4', '5', '6', '9', '10', '11', '12', '13', '23', '24',
       '25', '26', '27', '28', '32', '33', '37', '42', '43', '45', '46',
       '48', '49', '52', '53', '54', '55', '60'], dtype=object)

In [87]:
## filter only thre training environments 
partition_0 = partition_0[[i for i in partition_0.columns if any(i.startswith(site + '_') for site  in sites_u_train)]]

In [36]:
partition_0

,1_1_1_minor,1_1_1_major,1_1_2_minor,1_1_2_major,1_1_3_minor,1_1_3_major,1_1_4_minor,1_1_4_major,1_1_5_minor,1_1_5_major,...,60_1_8_minor,60_1_8_major,60_1_9_minor,60_1_9_major,60_1_10_minor,60_1_10_major,60_1_11_minor,60_1_11_major,60_1_12_minor,60_1_12_major
0,89,203,114,356,124,334,122,344,159,293,...,11,37,9,5,10,32,19,21,24,16
1,14,278,32,438,27,431,32,434,45,407,...,7,41,0,14,0,42,3,37,1,39
2,45,247,55,415,53,405,41,425,31,421,...,11,37,0,14,1,41,4,36,1,39
3,14,278,10,460,26,432,26,440,29,423,...,7,41,1,13,8,34,7,33,4,36
4,14,278,10,460,26,432,26,440,29,423,...,7,41,1,13,8,34,7,33,4,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,43,249,60,410,75,383,62,404,64,388,...,7,41,5,9,10,32,1,39,2,38
5220,91,201,200,270,146,312,169,297,157,295,...,9,39,5,9,20,22,14,26,23,17
5221,28,264,44,426,53,405,38,428,41,411,...,7,41,2,12,0,42,0,40,1,39
5222,98,194,201,269,154,304,170,296,159,293,...,9,39,5,9,21,21,14,26,23,17


In [37]:
minor_columns = partition_0.filter(like='minor')
major_columns = partition_0.filter(like='major')

In [38]:
# Properly calculate total allele counts by summing corresponding minor and major allele counts
total_allele_counts = minor_columns.values + major_columns.values

# Create a new DataFrame for total counts with proper column names
total_allele_counts_df = pd.DataFrame(total_allele_counts, columns=minor_columns.columns.str.replace('_minor', '_total'))

In [39]:

# Preparing data for binomial regression
# Minor allele counts as the response variable (successes)
minor_alleles = minor_columns.T.reset_index(drop=True)
major_columns = major_columns.T.reset_index(drop=True)

# Total allele counts as the number of trials (total attempts)
total_alleles = total_allele_counts_df.T.reset_index(drop=True)


In [40]:
successes = minor_alleles.iloc[:,1]
failures = major_columns.iloc[:,1]

In [41]:
# Running binomial regression for each SNP separately
coefficients_df = {}

for i in range(minor_alleles.shape[1]):
    # Prepare response (successes, failures) and predictors (environmental variable)
    successes = minor_alleles.iloc[:,i]
    failures = major_columns.iloc[:,i]
    # Set up the binomial regression model
    X = sm.add_constant(env_variable_train)  # Adding constant for intercept
    y = pd.concat([successes,failures],axis=1)
    # Fit the modelb
    model = sm.GLM(y, X, family=sm.families.Binomial())
    result = model.fit()

    # Extract slope (coefficient for environmental variable) and p-value
    slope = result.params[1]  # Coefficient for the environmental variable
    p_value = result.pvalues[1]  # P-value for the environmental variable
    intercept = result.params[0] 
    
    all_sites_pred_proba = []
    for site, row in env_site_scaled_unique.iterrows():
        # Predict the probability using the logistic model
        X_new = np.column_stack((np.ones(1), np.array([row['env_scaled']])))
        predicted_probability = result.predict(X_new)  # This gives the log-odds
        all_sites_pred_proba.append(predicted_probability[0])
    # Append the results to the DataFrame

    all_sites_pred_proba.append(slope)
    all_sites_pred_proba.append(p_value)
    all_sites_pred_proba.append(intercept)
    coefficients_df[i] = all_sites_pred_proba
    
    # Collect resul

In [42]:
coefficients_df = pd.DataFrame(coefficients_df).T

In [43]:
coefficients_df

,0,1,2,3
0,0.269733,0.006439,5.546146e-01,-0.993793
1,0.074699,0.081442,8.006620e-06,-2.489041
2,0.095246,-0.012440,4.509714e-01,-2.255414
3,0.080900,0.332737,6.253795e-84,-2.317382
4,0.080900,0.332737,6.253795e-84,-2.317382
...,...,...,...,...
5219,0.122976,-0.061937,2.937991e-05,-1.985542
5220,0.361217,0.107201,1.558036e-26,-0.533748
5221,0.079081,-0.174382,1.363996e-21,-2.514013
5222,0.366492,0.102454,1.635451e-24,-0.512566


In [44]:
sites_u_test

array(['57'], dtype=object)

In [88]:
coefficients_df.columns = [sites_u_test[0], 'slope', 'pvalue','intercept']

In [46]:
coefficients_df

,57,slope,pvalue,intercept
0,0.269733,0.006439,5.546146e-01,-0.993793
1,0.074699,0.081442,8.006620e-06,-2.489041
2,0.095246,-0.012440,4.509714e-01,-2.255414
3,0.080900,0.332737,6.253795e-84,-2.317382
4,0.080900,0.332737,6.253795e-84,-2.317382
...,...,...,...,...
5219,0.122976,-0.061937,2.937991e-05,-1.985542
5220,0.361217,0.107201,1.558036e-26,-0.533748
5221,0.079081,-0.174382,1.363996e-21,-2.514013
5222,0.366492,0.102454,1.635451e-24,-0.512566


In [47]:
import os

In [48]:
files = os.listdir('../baypass_first_gen/individual_gfiles/')

In [49]:
partitions = [int(file.split('_')[1].replace('.txt', '')) for file in files if '.txt' in file]

In [50]:
partitions.sort()

In [51]:
partitions[-1]

201

In [52]:
len(partitions)

202

In [53]:
pwd -P

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go'

In [54]:
import random
import subprocess

In [57]:
file_path = '../jacknife_first_gen/splits_samples_first_gen.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

In [58]:
# Loop through splits 0 to 31
for split in range(len(samples)):
    # Define the folder path
    folder_path = f'results/split_{split}'
    # Create the directory (makedirs allows creating intermediate directories if they don't exist)
    os.makedirs(folder_path, exist_ok=True)

In [60]:
pwd

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go'

In [91]:
# create sbatch files to submit on cedar server
shfiles = []
for split in range(len(samples)):
    for partition in partitions:
        seed = random.randint(1,100000000)
        file = f'shfiles/partition_{partition}_{split}.sh'
        cmd = f'python run_partition_binomial_reg_first_gen.py {partition} {split}'
        text = f'''#!/bin/bash
#SBATCH --job-name=run_partition_binomial_reg{partition}_{split}
#SBATCH --time=1:00:00  # Time limit set to 4 hours
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=30gb
#SBATCH --output=run_partition_binomial_reg_{partition}_{split}_%j.out
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

module load python/3.11_conda
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake
export LD_LIBRARY_PATH="/home/tbellagio/miniforge3/envs/run_baypass/lib:$LD_LIBRARY_PATH"
cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go
{cmd}

'''
        with open(file, 'w') as o:
            o.write("%s" % text)
        shfiles.append(file)

In [92]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 43078


CompletedProcess(args=['sbatch', 'shfiles/partition_0_0.sh'], returncode=0)

In [283]:
len(shfiles)

630

In [284]:
results/split_13/partition199
results/split_28/partition31.csv
results/split_29/partition0.csv

NameError: name 'split_13' is not defined

In [271]:
[i for i in shfiles if 'partition_199_13' in i]

['shfiles/partition_199_13.sh']

In [ ]:
## now run the shfiles
for shfile in shfiles:
    # Submit each sbatch script to the SLURM scheduler
    subprocess.run(["sbatch", shfile], check=True)

In [238]:
files = os.listdir('../baypass_lastgen/individual_gfiles_last_gen/')
loci_names = [file for file in files if 'loci' in file]

In [95]:
for split in range(len(samples)):
    print(split)
    print(len(os.listdir(f'results/split_{split}')))

0
203
1
202
2
202
3
202
4
202
5
202
6
202
7
202
8
202
9
202
10
202
11
202
12
202
13
202
14
202
15
202
16
202
17
202
18
202
19
202
20
202
21
202
22
202
23
202
24
202
25
202
26
202
27
202
28
202
29
202
30
202


In [97]:
for split in range(len(samples)):
    print(split)
    partitions_r = {}
    for i in range(len(partitions)):  # /baypass_first_gen/individual_gfiles/
        pickle_file_path = f'../baypass_first_gen/individual_gfiles/loci_partition_{i}'
        with open(pickle_file_path, 'rb') as file:
            loci_f = pickle.load(file)
        results = pd.read_csv(f'results/split_{split}/partition{i}.csv')
        results['snp_id'] = loci_f
        partitions_r[i] = results
    results = pd.concat(partitions_r).reset_index(drop=True)
    results.to_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv',index=None)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [66]:
all_loci = []
for i in loci_names:
    pickle_file_path = f'../baypass_lastgen/individual_gfiles_last_gen/{i}'
    with open(pickle_file_path, 'rb') as file:
        loci_f = pickle.load(file)
    all_loci.append(loci_f)

In [67]:
flattened_loci = [item for sublist in all_loci for item in sublist]

In [68]:
len(flattened_loci)

1048635

In [74]:
1054574 - 1055248

-674